In [2]:
!pip install fastf1
!pip install requests
!pip install datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 13.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import requests
import datetime
import random

# API keys and configuration
OPENWEATHER_API_KEY = "a8274b7959f3832231d9ccbdf579549c"  # Replace with your actual API key

# 2024 F1 Data: Races, Teams, Drivers
races_2024 = {
    "Bahrain": {
        "date": "2024-03-02",
        "location": {"lat": 26.0325, "lon": 50.5106},
        "track_difficulty": 7,
        "track_type": "smooth"
    },
    "Saudi Arabia": {
        "date": "2024-03-09",
        "location": {"lat": 21.6319, "lon": 39.1044},
        "track_difficulty": 8,
        "track_type": "street"
    },
    "Australia": {
        "date": "2024-03-24",
        "location": {"lat": -37.8497, "lon": 144.9680},
        "track_difficulty": 6,
        "track_type": "mixed"
    },
    "Japan": {
        "date": "2024-04-07",
        "location": {"lat": 34.8431, "lon": 136.5392},
        "track_difficulty": 9,
        "track_type": "technical"
    },
    "China": {
        "date": "2024-04-21",
        "location": {"lat": 31.3389, "lon": 121.2208},
        "track_difficulty": 7,
        "track_type": "mixed"
    },
    "Miami": {
        "date": "2024-05-05",
        "location": {"lat": 25.9581, "lon": -80.2389},
        "track_difficulty": 6,
        "track_type": "street"
    }
}

teams_2024 = {
    "Red Bull": {"performance": 95, "reliability": 90},
    "Mercedes": {"performance": 88, "reliability": 92},
    "Ferrari": {"performance": 90, "reliability": 85},
    "McLaren": {"performance": 92, "reliability": 88},
    "Aston Martin": {"performance": 83, "reliability": 87},
    "Alpine": {"performance": 80, "reliability": 83},
    "Williams": {"performance": 78, "reliability": 82},
    "RB": {"performance": 79, "reliability": 85},
    "Haas": {"performance": 77, "reliability": 80},
    "Sauber": {"performance": 75, "reliability": 81}
}

drivers_2024 = {
    "Max Verstappen": {"team": "Red Bull", "skill": 95, "wet_skill": 94, "consistency": 92},
    "Sergio Perez": {"team": "Red Bull", "skill": 86, "wet_skill": 82, "consistency": 80},
    "Lewis Hamilton": {"team": "Mercedes", "skill": 93, "wet_skill": 95, "consistency": 91},
    "George Russell": {"team": "Mercedes", "skill": 88, "wet_skill": 85, "consistency": 85},
    "Charles Leclerc": {"team": "Ferrari", "skill": 91, "wet_skill": 87, "consistency": 85},
    "Carlos Sainz": {"team": "Ferrari", "skill": 88, "wet_skill": 85, "consistency": 86},
    "Lando Norris": {"team": "McLaren", "skill": 90, "wet_skill": 88, "consistency": 87},
    "Oscar Piastri": {"team": "McLaren", "skill": 85, "wet_skill": 83, "consistency": 82},
    "Fernando Alonso": {"team": "Aston Martin", "skill": 92, "wet_skill": 93, "consistency": 89},
    "Lance Stroll": {"team": "Aston Martin", "skill": 82, "wet_skill": 80, "consistency": 78},
    "Esteban Ocon": {"team": "Alpine", "skill": 83, "wet_skill": 81, "consistency": 81},
    "Pierre Gasly": {"team": "Alpine", "skill": 84, "wet_skill": 82, "consistency": 80},
    "Alexander Albon": {"team": "Williams", "skill": 85, "wet_skill": 83, "consistency": 82},
    "Logan Sargeant": {"team": "Williams", "skill": 76, "wet_skill": 75, "consistency": 74},
    "Daniel Ricciardo": {"team": "RB", "skill": 84, "wet_skill": 83, "consistency": 81},
    "Yuki Tsunoda": {"team": "RB", "skill": 82, "wet_skill": 80, "consistency": 78},
    "Nico Hulkenberg": {"team": "Haas", "skill": 83, "wet_skill": 82, "consistency": 81},
    "Kevin Magnussen": {"team": "Haas", "skill": 82, "wet_skill": 81, "consistency": 79},
    "Valtteri Bottas": {"team": "Sauber", "skill": 84, "wet_skill": 82, "consistency": 83},
    "Zhou Guanyu": {"team": "Sauber", "skill": 78, "wet_skill": 77, "consistency": 76}
}

# Historical race results for training
race_results = {
    "Bahrain": [
        "Max Verstappen", "Sergio Perez", "Carlos Sainz", "Charles Leclerc", "George Russell",
        "Lewis Hamilton", "Lando Norris", "Oscar Piastri", "Fernando Alonso", "Lance Stroll"
    ],
    "Saudi Arabia": [
        "Max Verstappen", "Sergio Perez", "Charles Leclerc", "Oscar Piastri", "Fernando Alonso",
        "George Russell", "Lewis Hamilton", "Lando Norris", "Nico Hulkenberg", "Kevin Magnussen"
    ],
    "Australia": [
        "Carlos Sainz", "Charles Leclerc", "Lando Norris", "Oscar Piastri", "Fernando Alonso",
        "George Russell", "Lewis Hamilton", "Yuki Tsunoda", "Nico Hulkenberg", "Kevin Magnussen"
    ],
    "Japan": [
        "Max Verstappen", "Sergio Perez", "Carlos Sainz", "Charles Leclerc", "Lando Norris",
        "Fernando Alonso", "George Russell", "Oscar Piastri", "Lewis Hamilton", "Yuki Tsunoda"
    ],
    "China": [
        "Max Verstappen", "Lando Norris", "Sergio Perez", "Charles Leclerc", "Carlos Sainz",
        "George Russell", "Fernando Alonso", "Oscar Piastri", "Lewis Hamilton", "Nico Hulkenberg"
    ]
}

# Function to get historical weather data (simulated with OpenWeather API format)
def get_weather_data(lat, lon, date):
    # In a real implementation, you'd call the OpenWeather API
    # For demonstration, we'll simulate weather data

    # Simulated API call would be:
    # url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={int(datetime.datetime.strptime(date, '%Y-%m-%d').timestamp())}&appid={OPENWEATHER_API_KEY}"
    # response = requests.get(url)
    # data = response.json()

    # Simulate different weather conditions
    weather_conditions = [
        {"main": "Clear", "temp": round(random.uniform(20, 30), 1)},
        {"main": "Clouds", "temp": round(random.uniform(15, 25), 1)},
        {"main": "Rain", "temp": round(random.uniform(10, 20), 1)},
        {"main": "Drizzle", "temp": round(random.uniform(12, 22), 1)}
    ]

    return random.choice(weather_conditions)

# Function to prepare training data
def prepare_training_data(race_list):
    data = []

    for race_name in race_list:
        race_info = races_2024[race_name]
        weather = get_weather_data(
            race_info["location"]["lat"],
            race_info["location"]["lon"],
            race_info["date"]
        )

        is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
        temperature = weather["temp"]

        # Create features for each driver
        for driver_name, driver_info in drivers_2024.items():
            team_info = teams_2024[driver_info["team"]]

            # Calculate driver form (simulated - would be based on recent performances)
            recent_form = random.randint(70, 100)

            # Determine finishing position
            if driver_name in race_results[race_name]:
                position = race_results[race_name].index(driver_name) + 1
            else:
                position = random.randint(11, 20)  # If not in top 10, assign random position

            # Create feature vector
            feature = {
                "driver": driver_name,
                "race": race_name,
                "track_difficulty": race_info["track_difficulty"],
                "track_type": race_info["track_type"],
                "team_performance": team_info["performance"],
                "team_reliability": team_info["reliability"],
                "driver_skill": driver_info["skill"],
                "driver_wet_skill": driver_info["wet_skill"],
                "driver_consistency": driver_info["consistency"],
                "driver_form": recent_form,
                "is_wet": is_wet,
                "temperature": temperature,
                "position": position
            }

            data.append(feature)

    return pd.DataFrame(data)

# Function to create prediction features for a specific race
def prepare_prediction_data(race_name):
    data = []
    race_info = races_2024[race_name]

    weather = get_weather_data(
        race_info["location"]["lat"],
        race_info["location"]["lon"],
        race_info["date"]
    )

    is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
    temperature = weather["temp"]

    print(f"\nPredicting for {race_name} GP")
    print(f"Weather conditions: {weather['main']}, Temperature: {temperature}°C")
    print(f"Track difficulty: {race_info['track_difficulty']}/10, Track type: {race_info['track_type']}")

    for driver_name, driver_info in drivers_2024.items():
        team_info = teams_2024[driver_info["team"]]

        # Simulate recent form
        recent_form = random.randint(70, 100)

        # Create feature vector
        feature = {
            "driver": driver_name,
            "race": race_name,
            "track_difficulty": race_info["track_difficulty"],
            "track_type": race_info["track_type"],
            "team_performance": team_info["performance"],
            "team_reliability": team_info["reliability"],
            "driver_skill": driver_info["skill"],
            "driver_wet_skill": driver_info["wet_skill"],
            "driver_consistency": driver_info["consistency"],
            "driver_form": recent_form,
            "is_wet": is_wet,
            "temperature": temperature
        }

        data.append(feature)

    return pd.DataFrame(data)

# Build and train the model
def train_and_predict(training_races, prediction_race):
    # Prepare training data
    train_df = prepare_training_data(training_races)

    # Prepare feature columns
    feature_cols = [
        'track_difficulty', 'team_performance', 'team_reliability',
        'driver_skill', 'driver_wet_skill', 'driver_consistency',
        'driver_form', 'is_wet', 'temperature'
    ]

    # Encode categorical variables
    train_df_encoded = pd.get_dummies(train_df, columns=['track_type'])

    # Extract features for training
    X_train = train_df_encoded[feature_cols + [col for col in train_df_encoded.columns if col.startswith('track_type_')]]
    y_train = train_df_encoded['position']

    # Normalize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Prepare prediction data
    pred_df = prepare_prediction_data(prediction_race)
    pred_df_encoded = pd.get_dummies(pred_df, columns=['track_type'])

    # Make sure prediction data has same columns as training data
    for col in X_train.columns:
        if col not in pred_df_encoded.columns:
            pred_df_encoded[col] = 0

    X_pred = pred_df_encoded[X_train.columns]
    X_pred_scaled = scaler.transform(X_pred)

    # Make predictions
    predictions = model.predict(X_pred_scaled)

    # Create results dataframe
    results = pd.DataFrame({
        'driver': pred_df['driver'],
        'team': [drivers_2024[driver]['team'] for driver in pred_df['driver']],
        'predicted_position': predictions
    })

    # Sort by predicted position and return top 10
    return results.sort_values('predicted_position').head(10)

# Select training and prediction races
training_races = ["Bahrain", "Saudi Arabia", "Australia", "Japan"]
prediction_race = "China"  # Could be changed to any race in races_2024

# Train the model and make predictions
top_10_predictions = train_and_predict(training_races, prediction_race)

print("\nPredicted Top 10:")
for i, (index, row) in enumerate(top_10_predictions.iterrows(), 1):
    print(f"{i}. {row['driver']} ({row['team']})")


Predicting for China GP
Weather conditions: Clear, Temperature: 29.3°C
Track difficulty: 7/10, Track type: mixed

Predicted Top 10:
1. Max Verstappen (Red Bull)
2. Carlos Sainz (Ferrari)
3. Charles Leclerc (Ferrari)
4. Sergio Perez (Red Bull)
5. Lando Norris (McLaren)
6. Fernando Alonso (Aston Martin)
7. George Russell (Mercedes)
8. Lewis Hamilton (Mercedes)
9. Oscar Piastri (McLaren)
10. Lance Stroll (Aston Martin)


In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import fastf1
import matplotlib.pyplot as plt
import requests
import datetime
from datetime import datetime, timedelta
import warnings
import json
import os
import random

# Suppress warnings
warnings.filterwarnings('ignore')

# Create FastF1 cache directory if it doesn't exist
if not os.path.exists('f1_cache'):
    os.makedirs('f1_cache')

# Enable FastF1 cache
fastf1.Cache.enable_cache('f1_cache')

# API configuration
OPENWEATHER_API_KEY = "a8274b7959f3832231d9ccbdf579549c"  # Replace with your actual API key

# Define the race schedule for 2024
races_2024 = {
    "Bahrain": {
        "round": 1,
        "date": "2024-03-02",
        "location": {"lat": 26.0325, "lon": 50.5106},
        "track_difficulty": 7,
        "track_length": 5.412,
        "track_type": "smooth"
    },
    "Saudi Arabia": {
        "round": 2,
        "date": "2024-03-09",
        "location": {"lat": 21.6319, "lon": 39.1044},
        "track_difficulty": 8,
        "track_length": 6.174,
        "track_type": "street"
    },
    "Australia": {
        "round": 3,
        "date": "2024-03-24",
        "location": {"lat": -37.8497, "lon": 144.9680},
        "track_difficulty": 6,
        "track_length": 5.278,
        "track_type": "mixed"
    },
    "Japan": {
        "round": 4,
        "date": "2024-04-07",
        "location": {"lat": 34.8431, "lon": 136.5392},
        "track_difficulty": 9,
        "track_length": 5.807,
        "track_type": "technical"
    },
    "China": {
        "round": 5,
        "date": "2024-04-21",
        "location": {"lat": 31.3389, "lon": 121.2208},
        "track_difficulty": 7,
        "track_length": 5.451,
        "track_type": "mixed"
    },
    "Miami": {
        "round": 6,
        "date": "2024-05-05",
        "location": {"lat": 25.9581, "lon": -80.2389},
        "track_difficulty": 6,
        "track_length": 5.412,
        "track_type": "street"
    },
    "Emilia Romagna": {
        "round": 7,
        "date": "2024-05-19",
        "location": {"lat": 44.3439, "lon": 11.7167},
        "track_difficulty": 8,
        "track_length": 4.909,
        "track_type": "technical"
    },
    "Monaco": {
        "round": 8,
        "date": "2024-05-26",
        "location": {"lat": 43.7347, "lon": 7.4206},
        "track_difficulty": 10,
        "track_length": 3.337,
        "track_type": "street"
    },
    "Canada": {
        "round": 9,
        "date": "2024-06-09",
        "location": {"lat": 45.5000, "lon": -73.5228},
        "track_difficulty": 7,
        "track_length": 4.361,
        "track_type": "mixed"
    },
    "Spain": {
        "round": 10,
        "date": "2024-06-23",
        "location": {"lat": 41.5700, "lon": 2.2611},
        "track_difficulty": 7,
        "track_length": 4.675,
        "track_type": "smooth"
    },
    "Austria": {
        "round": 11,
        "date": "2024-06-30",
        "location": {"lat": 47.2197, "lon": 14.7647},
        "track_difficulty": 6,
        "track_length": 4.318,
        "track_type": "smooth"
    },
    "Great Britain": {
        "round": 12,
        "date": "2024-07-07",
        "location": {"lat": 52.0786, "lon": -1.0169},
        "track_difficulty": 8,
        "track_length": 5.891,
        "track_type": "technical"
    },
    "Hungary": {
        "round": 13,
        "date": "2024-07-21",
        "location": {"lat": 47.5829, "lon": 19.2486},
        "track_difficulty": 7,
        "track_length": 4.381,
        "track_type": "technical"
    },
    "Belgium": {
        "round": 14,
        "date": "2024-07-28",
        "location": {"lat": 50.4372, "lon": 5.9714},
        "track_difficulty": 9,
        "track_length": 7.004,
        "track_type": "technical"
    },
    "Netherlands": {
        "round": 15,
        "date": "2024-08-25",
        "location": {"lat": 52.3888, "lon": 4.5409},
        "track_difficulty": 7,
        "track_length": 4.259,
        "track_type": "smooth"
    },
    "Italy": {
        "round": 16,
        "date": "2024-09-01",
        "location": {"lat": 45.6206, "lon": 9.2811},
        "track_difficulty": 6,
        "track_length": 5.793,
        "track_type": "smooth"
    },
    "Azerbaijan": {
        "round": 17,
        "date": "2024-09-15",
        "location": {"lat": 40.3725, "lon": 49.8533},
        "track_difficulty": 8,
        "track_length": 6.003,
        "track_type": "street"
    },
    "Singapore": {
        "round": 18,
        "date": "2024-09-22",
        "location": {"lat": 1.2914, "lon": 103.8640},
        "track_difficulty": 9,
        "track_length": 5.063,
        "track_type": "street"
    },
    "USA": {
        "round": 19,
        "date": "2024-10-20",
        "location": {"lat": 30.1328, "lon": -97.6411},
        "track_difficulty": 7,
        "track_length": 5.513,
        "track_type": "mixed"
    },
    "Mexico": {
        "round": 20,
        "date": "2024-10-27",
        "location": {"lat": 19.4042, "lon": -99.0907},
        "track_difficulty": 6,
        "track_length": 4.304,
        "track_type": "smooth"
    },
    "Brazil": {
        "round": 21,
        "date": "2024-11-03",
        "location": {"lat": -23.7036, "lon": -46.6997},
        "track_difficulty": 7,
        "track_length": 4.309,
        "track_type": "technical"
    },
    "Las Vegas": {
        "round": 22,
        "date": "2024-11-23",
        "location": {"lat": 36.1147, "lon": -115.1728},
        "track_difficulty": 6,
        "track_length": 6.201,
        "track_type": "street"
    },
    "Qatar": {
        "round": 23,
        "date": "2024-12-01",
        "location": {"lat": 25.4900, "lon": 51.4542},
        "track_difficulty": 6,
        "track_length": 5.419,
        "track_type": "smooth"
    },
    "Abu Dhabi": {
        "round": 24,
        "date": "2024-12-08",
        "location": {"lat": 24.4672, "lon": 54.6031},
        "track_difficulty": 6,
        "track_length": 5.281,
        "track_type": "smooth"
    }
}

# Driver characteristics for 2024
drivers_2024 = {
    "VER": {"name": "Max Verstappen", "team": "Red Bull", "skill": 95, "wet_skill": 95, "consistency": 92},
    "PER": {"name": "Sergio Perez", "team": "Red Bull", "skill": 86, "wet_skill": 82, "consistency": 80},
    "HAM": {"name": "Lewis Hamilton", "team": "Mercedes", "skill": 93, "wet_skill": 95, "consistency": 91},
    "RUS": {"name": "George Russell", "team": "Mercedes", "skill": 88, "wet_skill": 85, "consistency": 86},
    "LEC": {"name": "Charles Leclerc", "team": "Ferrari", "skill": 91, "wet_skill": 87, "consistency": 85},
    "SAI": {"name": "Carlos Sainz", "team": "Ferrari", "skill": 88, "wet_skill": 85, "consistency": 86},
    "NOR": {"name": "Lando Norris", "team": "McLaren", "skill": 90, "wet_skill": 88, "consistency": 87},
    "PIA": {"name": "Oscar Piastri", "team": "McLaren", "skill": 86, "wet_skill": 83, "consistency": 82},
    "ALO": {"name": "Fernando Alonso", "team": "Aston Martin", "skill": 92, "wet_skill": 93, "consistency": 89},
    "STR": {"name": "Lance Stroll", "team": "Aston Martin", "skill": 82, "wet_skill": 80, "consistency": 78},
    "OCO": {"name": "Esteban Ocon", "team": "Alpine", "skill": 83, "wet_skill": 81, "consistency": 81},
    "GAS": {"name": "Pierre Gasly", "team": "Alpine", "skill": 84, "wet_skill": 82, "consistency": 80},
    "ALB": {"name": "Alexander Albon", "team": "Williams", "skill": 85, "wet_skill": 83, "consistency": 82},
    "SAR": {"name": "Logan Sargeant", "team": "Williams", "skill": 76, "wet_skill": 75, "consistency": 74},
    "RIC": {"name": "Daniel Ricciardo", "team": "RB", "skill": 84, "wet_skill": 83, "consistency": 81},
    "TSU": {"name": "Yuki Tsunoda", "team": "RB", "skill": 82, "wet_skill": 80, "consistency": 78},
    "HUL": {"name": "Nico Hulkenberg", "team": "Haas", "skill": 83, "wet_skill": 82, "consistency": 81},
    "MAG": {"name": "Kevin Magnussen", "team": "Haas", "skill": 82, "wet_skill": 81, "consistency": 79},
    "BOT": {"name": "Valtteri Bottas", "team": "Sauber", "skill": 84, "wet_skill": 82, "consistency": 83},
    "ZHO": {"name": "Zhou Guanyu", "team": "Sauber", "skill": 78, "wet_skill": 77, "consistency": 76}
}

# Team characteristics for 2024
teams_2024 = {
    "Red Bull": {"performance": 95, "reliability": 88, "wet_performance": 92},
    "Mercedes": {"performance": 88, "reliability": 92, "wet_performance": 90},
    "Ferrari": {"performance": 91, "reliability": 85, "wet_performance": 86},
    "McLaren": {"performance": 93, "reliability": 88, "wet_performance": 87},
    "Aston Martin": {"performance": 83, "reliability": 87, "wet_performance": 84},
    "Alpine": {"performance": 80, "reliability": 83, "wet_performance": 79},
    "Williams": {"performance": 78, "reliability": 82, "wet_performance": 80},
    "RB": {"performance": 79, "reliability": 85, "wet_performance": 81},
    "Haas": {"performance": 77, "reliability": 80, "wet_performance": 78},
    "Sauber": {"performance": 75, "reliability": 81, "wet_performance": 76}
}

class F1Predictor:
    def __init__(self, api_key, cache_dir='weather_cache'):
        self.api_key = api_key
        self.cache_dir = cache_dir
        self.weather_cache = {}
        self.lap_data = None
        self.performance_metrics = None
        self.model = None

        # Create cache directory if it doesn't exist
        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)

        # Load cached weather data if available
        self._load_weather_cache()

    def _load_weather_cache(self):
        """Load cached weather data from file"""
        cache_file = os.path.join(self.cache_dir, 'weather_cache.json')
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'r') as f:
                    self.weather_cache = json.load(f)
                print(f"Loaded weather data for {len(self.weather_cache)} races from cache")
            except Exception as e:
                print(f"Error loading weather cache: {e}")
                self.weather_cache = {}

    def _save_weather_cache(self):
        """Save weather data to cache file"""
        cache_file = os.path.join(self.cache_dir, 'weather_cache.json')
        try:
            with open(cache_file, 'w') as f:
                json.dump(self.weather_cache, f)
        except Exception as e:
            print(f"Error saving weather cache: {e}")

    def get_weather_data(self, race_name):
        """Get weather data for a specific race, using cache if available"""
        if race_name in self.weather_cache:
            print(f"Using cached weather data for {race_name}")
            return self.weather_cache[race_name]

        race_info = races_2024[race_name]
        lat = race_info["location"]["lat"]
        lon = race_info["location"]["lon"]
        date = race_info["date"]

        weather = self._fetch_openweather_data(lat, lon, date)

        self.weather_cache[race_name] = weather
        self._save_weather_cache()

        return weather

    def _fetch_openweather_data(self, lat, lon, date_str):
        """Fetch weather data from OpenWeather API 2.5"""
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        today = datetime.now()
        days_diff = (date_obj - today).days

        print(f"Fetching weather data for {date_str} at {lat}, {lon}")

        if days_diff <= 5 and days_diff >= 0:
            url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={self.api_key}&units=metric"
            try:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()

                target_time = date_obj.replace(hour=12, minute=0, second=0)
                forecasts = data.get('list', [])
                closest_forecast = min(
                    forecasts,
                    key=lambda x: abs(datetime.fromtimestamp(x['dt']) - target_time),
                    default=None
                )

                if closest_forecast:
                    return {
                        'main': closest_forecast['weather'][0]['main'],
                        'description': closest_forecast['weather'][0]['description'],
                        'temp': closest_forecast['main']['temp'],
                        'humidity': closest_forecast['main']['humidity'],
                        'wind_speed': closest_forecast['wind']['speed'],
                        'pressure': closest_forecast['main'].get('pressure', 1013),
                        'rain_1h': closest_forecast.get('rain', {}).get('3h', 0) / 3,
                        'clouds': closest_forecast['clouds']['all']
                    }
            except Exception as e:
                print(f"Error fetching forecast: {e}")

        return self._simulate_weather_data(date_obj.month)

    def _simulate_weather_data(self, month):
        """Generate simulated weather data based on month"""
        season_data = {
            12: {"temp_range": (0, 15), "rain_prob": 0.4, "clouds_prob": 0.7},
            1: {"temp_range": (0, 15), "rain_prob": 0.4, "clouds_prob": 0.7},
            2: {"temp_range": (0, 15), "rain_prob": 0.4, "clouds_prob": 0.7},
            3: {"temp_range": (5, 20), "rain_prob": 0.3, "clouds_prob": 0.5},
            4: {"temp_range": (10, 25), "rain_prob": 0.3, "clouds_prob": 0.5},
            5: {"temp_range": (15, 30), "rain_prob": 0.2, "clouds_prob": 0.4},
            6: {"temp_range": (20, 35), "rain_prob": 0.1, "clouds_prob": 0.3},
            7: {"temp_range": (20, 35), "rain_prob": 0.1, "clouds_prob": 0.3},
            8: {"temp_range": (20, 35), "rain_prob": 0.1, "clouds_prob": 0.3},
            9: {"temp_range": (15, 30), "rain_prob": 0.2, "clouds_prob": 0.4},
            10: {"temp_range": (10, 25), "rain_prob": 0.3, "clouds_prob": 0.5},
            11: {"temp_range": (5, 20), "rain_prob": 0.4, "clouds_prob": 0.6},
        }

        data = season_data.get(month, {"temp_range": (15, 25), "rain_prob": 0.3, "clouds_prob": 0.5})

        rain_check = np.random.random()
        cloud_check = np.random.random()

        if rain_check < data["rain_prob"]:
            weather_main = "Rain" if rain_check < data["rain_prob"]/2 else "Drizzle"
            rain_amount = np.random.uniform(0.5, 10)
        else:
            rain_amount = 0
            weather_main = "Clouds" if cloud_check < data["clouds_prob"] else "Clear"

        print("Using simulated weather data")
        return {
            'main': weather_main,
            'description': weather_main.lower(),
            'temp': round(np.random.uniform(data["temp_range"][0], data["temp_range"][1]), 1),
            'humidity': round(np.random.uniform(40, 95)),
            'wind_speed': round(np.random.uniform(1, 15), 1),
            'pressure': round(np.random.uniform(1000, 1025)),
            'rain_1h': rain_amount,
            'clouds': round(np.random.uniform(0, 100)) if weather_main in ["Clouds", "Rain", "Drizzle"] else 0
        }

    def fetch_historical_lap_data(self, year=2024, rounds=None):
        """Fetch historical lap time and qualifying data using FastF1"""
        if rounds is None:
            rounds = [1, 2, 3, 4, 5]

        print(f"Fetching historical lap data for {year}, rounds {rounds}")
        all_lap_data = []

        driver_mapping = {'BEA': 'SAI'}

        for race_round in rounds:
            try:
                race = fastf1.get_session(year, race_round, 'R')
                race.load()
                print(f"Loaded race data for {year} Round {race_round}: {race.event['EventName']}")

                laps = race.laps
                laps['Year'] = year
                laps['Round'] = race_round
                laps['EventName'] = race.event['EventName']

                laps['Driver'] = laps['Driver'].replace(driver_mapping)

                qualy = fastf1.get_session(year, race_round, 'Q')
                qualy.load()
                print(f"Loaded qualifying data for {year} Round {race_round}")
                qualy_results = qualy.results[['DriverNumber', 'Position']].rename(
                    columns={'Position': 'QualifyingPosition'}
                )
                laps = laps.merge(qualy_results, on='DriverNumber', how='left')

                all_lap_data.append(laps)

            except Exception as e:
                print(f"Error loading data for {year} Round {race_round}: {e}")
                continue

        if all_lap_data:
            self.lap_data = pd.concat(all_lap_data)
            unique_drivers = len(self.lap_data['Driver'].unique())
            print(f"Successfully loaded {len(all_lap_data)} races with {unique_drivers} unique drivers")
            if unique_drivers < 18:
                print("Warning: Insufficient driver data, using fallback")
                self.lap_data = None
                return None
            return self.lap_data
        else:
            print("No lap data available, using fallback")
            self.lap_data = None
            return None

    def process_lap_data(self):
        """Process lap data to extract driver performance metrics"""
        if self.lap_data is None or self.lap_data.empty:
            print("No lap data to process")
            self.performance_metrics = None
            return None

        metrics = []
        valid_drivers = set(drivers_2024.keys())
        grouped = self.lap_data[self.lap_data['Driver'].isin(valid_drivers)].groupby(['EventName', 'Driver'])

        for (event, driver_code), group in grouped:
            valid_laps = group[group['LapTime'].notna()]
            if valid_laps.empty:
                continue

            valid_laps['LapTimeSeconds'] = valid_laps['LapTime'].dt.total_seconds()

            median_lap = valid_laps['LapTimeSeconds'].median()
            fastest_lap = valid_laps['LapTimeSeconds'].min()
            lap_std = valid_laps['LapTimeSeconds'].std()

            if pd.isna(lap_std):
                lap_std = 0

            consistency = max(0, 100 - (lap_std * 5))

            if 'Position' in valid_laps.columns:
                final_pos = valid_laps.iloc[-1]['Position']
                start_pos = valid_laps.iloc[0]['Position']
                pos_change = start_pos - final_pos
            else:
                final_pos = np.nan
                start_pos = np.nan
                pos_change = 0

            qualifying_pos = valid_laps['QualifyingPosition'].iloc[0] if 'QualifyingPosition' in valid_laps.columns and not valid_laps['QualifyingPosition'].isna().all() else np.nan

            driver_metrics = {
                'Driver': driver_code,
                'Event': event,
                'Round': valid_laps['Round'].iloc[0],
                'MedianLapTime': median_lap,
                'FastestLapTime': fastest_lap,
                'LapTimeStd': lap_std,
                'Consistency': consistency,
                'FinalPosition': final_pos,
                'StartingPosition': start_pos,
                'QualifyingPosition': qualifying_pos,
                'PositionChange': pos_change,
                'LapsCompleted': len(valid_laps)
            }

            metrics.append(driver_metrics)

        self.performance_metrics = pd.DataFrame(metrics)
        if self.performance_metrics.empty:
            print("No valid performance metrics processed")
            return None
        return self.performance_metrics

    def _calculate_recent_form(self):
        """Calculate weighted average position in last 3 races, blended with skill"""
        race_results = {
            "Bahrain": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "RUS": 5,
                "HAM": 6, "NOR": 7, "PIA": 8, "ALO": 9, "STR": 10,
                "OCO": 11, "GAS": 12, "ALB": 13, "MAG": 14, "TSU": 15,
                "RIC": 16, "HUL": 17, "ZHO": 18, "SAR": 19, "BOT": 20
            },
            "Saudi Arabia": {
                "VER": 1, "PER": 2, "LEC": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "SAI": 7, "HAM": 8, "HUL": 9, "MAG": 10,
                "ALB": 11, "STR": 12, "RIC": 13, "GAS": 14, "OCO": 15,
                "SAR": 16, "TSU": 17, "BOT": 18, "ZHO": 19, "NOR": 20
            },
            "Australia": {
                "SAI": 1, "LEC": 2, "NOR": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "HAM": 7, "TSU": 8, "HUL": 9, "MAG": 10,
                "RIC": 11, "PER": 12, "ALB": 13, "OCO": 14, "BOT": 15,
                "ZHO": 16, "GAS": 17, "STR": 18, "VER": 19, "SAR": 20
            },
            "Japan": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "NOR": 5,
                "ALO": 6, "RUS": 7, "PIA": 8, "HAM": 9, "TSU": 10,
                "HUL": 11, "STR": 12, "RIC": 13, "OCO": 14, "ALB": 15,
                "MAG": 16, "GAS": 17, "BOT": 18, "ZHO": 19, "SAR": 20
            },
            "China": {
                "VER": 1, "NOR": 2, "PER": 3, "LEC": 4, "SAI": 5,
                "RUS": 6, "ALO": 7, "PIA": 8, "HAM": 9, "HUL": 10,
                "OCO": 11, "ALB": 12, "RIC": 13, "STR": 14, "GAS": 15,
                "MAG": 16, "BOT": 17, "ZHO": 18, "TSU": 19, "SAR": 20
            }
        }

        recent_form = {}
        race_order = ["China", "Japan", "Australia", "Saudi Arabia", "Bahrain"]

        for driver_code in drivers_2024:
            skill = drivers_2024[driver_code]["skill"]
            skill_based_form = 21 - (skill / 10)

            positions = []
            if self.performance_metrics is not None and not self.performance_metrics.empty:
                driver_races = self.performance_metrics[self.performance_metrics['Driver'] == driver_code]
                sorted_races = driver_races.sort_values('Round', ascending=False).head(3)
                positions = sorted_races['FinalPosition'].dropna().tolist()

            if len(positions) < 2:
                positions = []
                for race in race_order[:3]:
                    if race in race_results and driver_code in race_results[race]:
                        positions.append(race_results[race][driver_code])

            if not positions:
                recent_form[driver_code] = skill_based_form
                continue

            capped_positions = np.clip(positions, a_min=None, a_max=15)
            weights = [0.6, 0.3, 0.1][:len(capped_positions)]
            weights += [0] * (3 - len(weights))
            weighted_avg = np.average(capped_positions, weights=weights)
            blended_form = 0.7 * weighted_avg + 0.3 * skill_based_form
            blended_form = max(blended_form, skill_based_form * 0.98)
            recent_form[driver_code] = blended_form if not np.isnan(blended_form) else 10

        return recent_form

    def _calculate_track_type_performance(self, target_track_type):
        """Calculate average position on tracks of the same type, blended with team"""
        if self.performance_metrics is None or self.performance_metrics.empty:
            return {driver: 10 for driver in drivers_2024}

        monaco_skill_adjust = {
            'LEC': 0.8, 'VER': 0.85, 'NOR': 0.9, 'HAM': 0.9, 'SAI': 0.9,
            'PER': 0.95, 'RUS': 0.95, 'PIA': 0.98, 'ALO': 0.98, 'RIC': 0.98,
            'OCO': 1.0, 'GAS': 1.0, 'ALB': 1.0, 'HUL': 1.02, 'BOT': 1.02,
            'TSU': 1.05, 'STR': 1.1, 'MAG': 1.1, 'ZHO': 1.1, 'SAR': 1.15
        }

        track_type_perf = {}
        relevant_races = [name for name, info in races_2024.items() if info['track_type'] == target_track_type]

        for driver_code in drivers_2024:
            driver_races = self.performance_metrics[
                (self.performance_metrics['Driver'] == driver_code) &
                (self.performance_metrics['Event'].isin(relevant_races))
            ]
            valid_positions = driver_races['FinalPosition'].dropna()
            team_perf = teams_2024[drivers_2024[driver_code]["team"]]["performance"]

            if len(valid_positions) < 3:
                base_perf = 21 - (team_perf / 10)
                if target_track_type == 'street':
                    base_perf *= monaco_skill_adjust.get(driver_code, 1.0)
                track_type_perf[driver_code] = min(base_perf, 15)
                continue

            team_weight = team_perf / 100 * 0.4 + 0.8
            avg_position = valid_positions.mean() / team_weight
            blended_perf = 0.7 * avg_position + 0.3 * (21 - (team_perf / 10))
            if target_track_type == 'street':
                blended_perf *= monaco_skill_adjust.get(driver_code, 1.0)
            track_type_perf[driver_code] = min(blended_perf, 15) if not np.isnan(blended_perf) else 10

        return track_type_perf

    def prepare_training_data(self, race_rounds=None):
        """Prepare training data from lap metrics and race information"""
        if race_rounds is None:
            race_rounds = [1, 2, 3, 4, 5]

        race_names = [race for race, info in races_2024.items() if info["round"] in race_rounds]

        print(f"Preparing training data for races: {', '.join(race_names)}")

        if self.performance_metrics is None and self.lap_data is not None:
            self.process_lap_data()

        if self.performance_metrics is None or self.performance_metrics.empty:
            return self._create_basic_training_data(race_names)

        training_data = []
        recent_form = self._calculate_recent_form()

        for race_name in race_names:
            race_info = races_2024[race_name]
            weather = self.get_weather_data(race_name)
            track_type_perf = self._calculate_track_type_performance(race_info["track_type"])

            is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
            is_cloudy = 1 if weather["main"] == "Clouds" else 0
            temperature = weather["temp"]
            humidity = weather["humidity"]
            wind_speed = weather["wind_speed"]

            for driver_code, driver_info in drivers_2024.items():
                team_info = teams_2024[driver_info["team"]]

                driver_race_metrics = self.performance_metrics[
                    (self.performance_metrics['Driver'] == driver_code) &
                    (self.performance_metrics['Event'].str.contains(race_name))
                ]

                if not driver_race_metrics.empty:
                    consistency = driver_race_metrics['Consistency'].iloc[0]
                    finish_position = driver_race_metrics['FinalPosition'].iloc[0]
                    qualifying_position = driver_race_metrics['QualifyingPosition'].iloc[0] if 'QualifyingPosition' in driver_race_metrics.columns and not pd.isna(driver_race_metrics['QualifyingPosition'].iloc[0]) else 10
                    if pd.isna(finish_position):
                        finish_position = 20
                else:
                    consistency = driver_info["consistency"]
                    finish_position = 20
                    qualifying_position = 10

                features = {
                    'Driver': driver_code,
                    'DriverName': driver_info["name"],
                    'Race': race_name,
                    'TrackDifficulty': race_info["track_difficulty"],
                    'TrackLength': race_info["track_length"],
                    'TrackType': race_info["track_type"],
                    'TeamPerformance': team_info["performance"],
                    'TeamReliability': team_info["reliability"],
                    'TeamWetPerformance': team_info["wet_performance"],
                    'DriverSkill': driver_info["skill"],
                    'DriverWetSkill': driver_info["wet_skill"],
                    'DriverConsistency': consistency,
                    'RecentForm': recent_form.get(driver_code, 10),
                    'TrackTypePerformance': track_type_perf.get(driver_code, 10),
                    'QualifyingPosition': qualifying_position,
                    'IsWet': is_wet,
                    'IsCloudy': is_cloudy,
                    'Temperature': temperature,
                    'Humidity': humidity,
                    'WindSpeed': wind_speed,
                    'Position': finish_position
                }

                training_data.append(features)

        return pd.DataFrame(training_data)

    def _create_basic_training_data(self, race_names):
        """Create basic training data without lap metrics"""
        print("Creating basic training data from known results")
        training_data = []

        race_results = {
            "Bahrain": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "RUS": 5,
                "HAM": 6, "NOR": 7, "PIA": 8, "ALO": 9, "STR": 10,
                "OCO": 11, "GAS": 12, "ALB": 13, "MAG": 14, "TSU": 15,
                "RIC": 16, "HUL": 17, "ZHO": 18, "SAR": 19, "BOT": 20
            },
            "Saudi Arabia": {
                "VER": 1, "PER": 2, "LEC": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "SAI": 7, "HAM": 8, "HUL": 9, "MAG": 10,
                "ALB": 11, "STR": 12, "RIC": 13, "GAS": 14, "OCO": 15,
                "SAR": 16, "TSU": 17, "BOT": 18, "ZHO": 19, "NOR": 20
            },
            "Australia": {
                "SAI": 1, "LEC": 2, "NOR": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "HAM": 7, "TSU": 8, "HUL": 9, "MAG": 10,
                "RIC": 11, "PER": 12, "ALB": 13, "OCO": 14, "BOT": 15,
                "ZHO": 16, "GAS": 17, "STR": 18, "VER": 19, "SAR": 20
            },
            "Japan": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "NOR": 5,
                "ALO": 6, "RUS": 7, "PIA": 8, "HAM": 9, "TSU": 10,
                "HUL": 11, "STR": 12, "RIC": 13, "OCO": 14, "ALB": 15,
                "MAG": 16, "GAS": 17, "BOT": 18, "ZHO": 19, "SAR": 20
            },
            "China": {
                "VER": 1, "NOR": 2, "PER": 3, "LEC": 4, "SAI": 5,
                "RUS": 6, "ALO": 7, "PIA": 8, "HAM": 9, "HUL": 10,
                "OCO": 11, "ALB": 12, "RIC": 13, "STR": 14, "GAS": 15,
                "MAG": 16, "BOT": 17, "ZHO": 18, "TSU": 19, "SAR": 20
            }
        }

        recent_form = self._calculate_recent_form()

        for race_name in race_names:
            race_info = races_2024[race_name]
            weather = self.get_weather_data(race_name)
            track_type_perf = self._calculate_track_type_performance(race_info["track_type"])

            is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
            is_cloudy = 1 if weather["main"] == "Clouds" else 0
            temperature = weather["temp"]
            humidity = weather["humidity"]
            wind_speed = weather["wind_speed"]

            for driver_code, driver_info in drivers_2024.items():
                team_info = teams_2024[driver_info["team"]]

                if race_name in race_results and driver_code in race_results[race_name]:
                    position = race_results[race_name][driver_code]
                    qualifying_position = max(1, min(20, position + np.random.randint(-2, 3)))
                else:
                    position = 21 - round((team_info["performance"] + driver_info["skill"]) / 20)
                    position = max(1, min(20, position))
                    qualifying_position = max(1, min(20, position + np.random.randint(-2, 3)))

                features = {
                    'Driver': driver_code,
                    'DriverName': driver_info["name"],
                    'Race': race_name,
                    'TrackDifficulty': race_info["track_difficulty"],
                    'TrackLength': race_info["track_length"],
                    'TrackType': race_info["track_type"],
                    'TeamPerformance': team_info["performance"],
                    'TeamReliability': team_info["reliability"],
                    'TeamWetPerformance': team_info["wet_performance"],
                    'DriverSkill': driver_info["skill"],
                    'DriverWetSkill': driver_info["wet_skill"],
                    'DriverConsistency': driver_info["consistency"],
                    'RecentForm': recent_form.get(driver_code, 10),
                    'TrackTypePerformance': track_type_perf.get(driver_code, 10),
                    'QualifyingPosition': qualifying_position,
                    'IsWet': is_wet,
                    'IsCloudy': is_cloudy,
                    'Temperature': temperature,
                    'Humidity': humidity,
                    'WindSpeed': wind_speed,
                    'Position': position
                }

                training_data.append(features)

        return pd.DataFrame(training_data)

    def prepare_prediction_data(self, race_name):
        """Prepare prediction data for a specific race"""
        print(f"Preparing prediction data for {race_name}")
        race_info = races_2024[race_name]

        weather = self.get_weather_data(race_name)

        is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
        is_cloudy = 1 if weather["main"] == "Clouds" else 0
        temperature = weather["temp"]
        humidity = weather["humidity"]
        wind_speed = weather["wind_speed"]

        print(f"\nPrediction for: {race_name} Grand Prix")
        print(f"Date: {race_info['date']}")
        print(f"Weather: {weather['main']} ({weather['description']})")
        print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind: {wind_speed} m/s")
        print(f"Track difficulty: {race_info['track_difficulty']}/10, Type: {race_info['track_type']}")

        prediction_data = []
        recent_form = self._calculate_recent_form()
        track_type_perf = self._calculate_track_type_performance(race_info["track_type"])

        for driver_code, driver_info in drivers_2024.items():
            team_info = teams_2024[driver_info["team"]]

            # Calculate base scores for all drivers
            qual_scores = []
            for dc, di in drivers_2024.items():
                ti = teams_2024[di["team"]]
                score = (ti["performance"] * 0.3 + di["skill"] * 0.7) / 5
                qual_scores.append((dc, score, di["skill"]))
            # Sort by score (descending) and skill (descending) for tiebreaker
            qual_scores.sort(key=lambda x: (x[1], x[2]), reverse=True)
            # Assign positions based on rank
            qualifying_position = next(i + 1 for i, (dc, _, _) in enumerate(qual_scores) if dc == driver_code)
            # Add slight randomization for realism
            qualifying_position = max(1, min(20, qualifying_position + np.random.randint(-1, 2)))

            if driver_code in ['VER', 'LEC', 'SAI', 'RUS', 'NOR', 'PIA', 'SAR', 'TSU']:
                print(f"Driver {driver_info['name']}: QualifyingPosition={qualifying_position}, RecentForm={recent_form.get(driver_code, 10):.2f}, TrackTypePerformance={track_type_perf.get(driver_code, 10):.2f}, DriverSkill={driver_info['skill']}, DriverConsistency={driver_info['consistency']}")

            features = {
                'Driver': driver_code,
                'DriverName': driver_info["name"],
                'Race': race_name,
                'TrackDifficulty': race_info["track_difficulty"],
                'TrackLength': race_info["track_length"],
                'TrackType': race_info["track_type"],
                'TeamPerformance': team_info["performance"],
                'TeamReliability': team_info["reliability"],
                'TeamWetPerformance': team_info["wet_performance"],
                'DriverSkill': driver_info["skill"],
                'DriverWetSkill': driver_info["wet_skill"],
                'DriverConsistency': driver_info["consistency"],
                'RecentForm': recent_form.get(driver_code, 10),
                'TrackTypePerformance': track_type_perf.get(driver_code, 10),
                'QualifyingPosition': qualifying_position,
                'IsWet': is_wet,
                'IsCloudy': is_cloudy,
                'Temperature': temperature,
                'Humidity': humidity,
                'WindSpeed': wind_speed
            }

            prediction_data.append(features)

        return pd.DataFrame(prediction_data)

    def train_model(self, training_data):
        """Train the Random Forest model"""
        print("Training Random Forest model...")

        numeric_features = [
            'TrackDifficulty', 'TrackLength', 'TeamPerformance',
            'TeamReliability', 'TeamWetPerformance', 'DriverSkill',
            'DriverWetSkill', 'DriverConsistency', 'RecentForm',
            'TrackTypePerformance', 'QualifyingPosition', 'Temperature',
            'Humidity', 'WindSpeed'
        ]
        categorical_features = ['TrackType', 'IsWet', 'IsCloudy']

        X = training_data.drop(['Driver', 'DriverName', 'Race', 'Position'], axis=1, errors='ignore')
        y = training_data['Position']

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), [f for f in numeric_features if f not in ['DriverSkill', 'TeamPerformance', 'QualifyingPosition', 'DriverWetSkill', 'TeamWetPerformance', 'TrackTypePerformance', 'DriverConsistency', 'RecentForm']]),
                ('skill', StandardScaler(), ['DriverSkill']),
                ('team', StandardScaler(), ['TeamPerformance']),
                ('qualy', StandardScaler(), ['QualifyingPosition']),
                ('wet_skill', StandardScaler(), ['DriverWetSkill']),
                ('wet_team', StandardScaler(), ['TeamWetPerformance']),
                ('track_type', StandardScaler(), ['TrackTypePerformance']),
                ('consistency', StandardScaler(), ['DriverConsistency']),
                ('form', StandardScaler(), ['RecentForm']),
                ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
            ],
            transformer_weights={'num': 1.0, 'skill': 1.2, 'team': 1.2, 'qualy': 2.5, 'wet_skill': 0.5, 'wet_team': 0.5, 'track_type': 1.5, 'consistency': 1.3, 'form': 1.5, 'cat': 1.0}
        )

        self.model = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', RandomForestRegressor(
                n_estimators=200,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42
            ))
        ])

        self.model.fit(X, y)
        print("Model training completed")

        return self.model

    def predict_race(self, race_name):
        """Predict race results for a given race"""
        if self.model is None:
            raise ValueError("Model not trained. Please train the model first.")

        pred_data = self.prepare_prediction_data(race_name)

        X_pred = pred_data.drop(['Driver', 'DriverName', 'Race'], axis=1)
        predictions = self.model.predict(X_pred)

        results = pd.DataFrame({
            'Driver': pred_data['Driver'],
            'DriverName': pred_data['DriverName'],
            'Team': [drivers_2024[driver]['team'] for driver in pred_data['Driver']],
            'PredictedPosition': predictions
        })

        results = results.sort_values('PredictedPosition')
        results['PredictedPosition'] = range(1, len(results) + 1)

        return results

    def visualize_predictions(self, race_name, results):
        """Visualize the predicted race results"""
        plt.figure(figsize=(12, 8))

        top_10 = results.head(10)

        bars = plt.bar(top_10['DriverName'], top_10['PredictedPosition'])

        plt.title(f'Predicted Race Results for {race_name} Grand Prix')
        plt.xlabel('Driver')
        plt.ylabel('Predicted Position')
        plt.xticks(rotation=45, ha='right')

        plt.gca().invert_yaxis()
        plt.yticks(range(1, 11))
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)

        team_colors = {
            'Red Bull': '#0600EF',
            'Mercedes': '#00D2BE',
            'Ferrari': '#DC0000',
            'McLaren': '#FF8700',
            'Aston Martin': '#00665E',
            'Alpine': '#0090FF',
            'Williams': '#005AFF',
            'RB': '#2B4562',
            'Haas': '#FFFFFF',
            'Sauber': '#0066FF'
        }

        for bar, team in zip(bars, top_10['Team']):
            bar.set_color(team_colors.get(team, '#888888'))

        plt.tight_layout()
        plt.savefig('race_prediction.png')
        plt.close()

    def run_prediction_pipeline(self, train_rounds=None):
        """Run the complete prediction pipeline for a randomly selected race"""
        print(f"\nRunning prediction pipeline...")

        if train_rounds is None:
            train_rounds = [1, 2, 3, 4, 5]

        # Select a random race not in training rounds
        available_races = [name for name, info in races_2024.items() if info["round"] not in train_rounds]
        race_name = random.choice(available_races)
        print(f"Selected race: {race_name} (Round {races_2024[race_name]['round']})")

        self.fetch_historical_lap_data(rounds=train_rounds)
        training_data = self.prepare_training_data(train_rounds)

        self.train_model(training_data)

        results = self.predict_race(race_name)

        print(f"\nPredicted Results for {race_name} Grand Prix:")
        print("\nPosition | Driver | Team")
        print("-" * 40)
        for idx, row in results.head(10).iterrows():
            print(f"{row['PredictedPosition']:2d} | {row['DriverName']:<20} | {row['Team']}")

        self.visualize_predictions(race_name, results)

        return results

if __name__ == "__main__":
    predictor = F1Predictor(api_key=OPENWEATHER_API_KEY)

    training_rounds = [1, 2, 3, 4, 5]

    try:
        results = predictor.run_prediction_pipeline(training_rounds)
    except Exception as e:
        print(f"Error running prediction pipeline: {e}")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
INFO:fastf1.fastf1.req:Using cached data for timing_app_data
core         

Loaded weather data for 7 races from cache

Running prediction pipeline...
Selected race: Canada (Round 9)
Fetching historical lap data for 2024, rounds [1, 2, 3, 4, 5]


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            I

Loaded race data for 2024 Round 1: Bahrain Grand Prix


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            I

...

Loaded qualifying data for 2024 Round 4


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:08.313000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '11', '16', '55', '63', '14', '81', '44', '27', '31', '23', '10', '24', '18', '20', '2', '3', '22', '77']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '4', '11', '16', '55', '63', '14', '81', '44', '27', '31', '23', '10', '24', '18', '20', '2', '3', '22', '77']
core           INFO 	Loading data for Chinese Grand Prix - Qualify

Loaded race data for 2024 Round 5: Chinese Grand Prix


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '4', '81', '16', '55', '63', '27', '77', '18', '3', '31', '23', '10', '24', '20', '44', '22', '2']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '11', '14', '4', '81', '16', '55', '63', '27', '77', '18', '3', '31', '23', '10', '24', '20', '44', '22', '2']


Loaded qualifying data for 2024 Round 5
Successfully loaded 5 races with 20 unique drivers
Preparing training data for races: Bahrain, Saudi Arabia, Australia, Japan, China
Using cached weather data for Bahrain
Using cached weather data for Saudi Arabia
Using cached weather data for Australia
Using cached weather data for Japan
Using cached weather data for China
Training Random Forest model...
Model training completed
Preparing prediction data for Canada
Fetching weather data for 2024-06-09 at 45.5, -73.5228
Using simulated weather data

Prediction for: Canada Grand Prix
Date: 2024-06-09
Weather: Clear (clear)
Temperature: 24.6°C, Humidity: 56%, Wind: 14.2 m/s
Track difficulty: 7/10, Type: mixed
Driver Max Verstappen: QualifyingPosition=2, RecentForm=11.27, TrackTypePerformance=11.50, DriverSkill=95, DriverConsistency=92
Driver George Russell: QualifyingPosition=9, RecentForm=11.96, TrackTypePerformance=12.20, DriverSkill=88, DriverConsistency=86
Driver Charles Leclerc: QualifyingPosi

In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import fastf1
import matplotlib.pyplot as plt
import requests
import datetime
from datetime import datetime, timedelta
import warnings
import json
import os
import random

# Suppress warnings
warnings.filterwarnings('ignore')

# Create FastF1 cache directory if it doesn't exist
if not os.path.exists('f1_cache'):
    os.makedirs('f1_cache')

# Enable FastF1 cache
fastf1.Cache.enable_cache('f1_cache')

# API configuration
OPENWEATHER_API_KEY = "a8274b7959f3832231d9ccbdf579549c"  # Replace with your actual API key

# Define the race schedule for 2024
races_2024 = {
    "Bahrain": {
        "round": 1,
        "date": "2024-03-02",
        "location": {"lat": 26.0325, "lon": 50.5106},
        "track_difficulty": 7,
        "track_length": 5.412,
        "track_type": "smooth"
    },
    "Saudi Arabia": {
        "round": 2,
        "date": "2024-03-09",
        "location": {"lat": 21.6319, "lon": 39.1044},
        "track_difficulty": 8,
        "track_length": 6.174,
        "track_type": "street"
    },
    "Australia": {
        "round": 3,
        "date": "2024-03-24",
        "location": {"lat": -37.8497, "lon": 144.9680},
        "track_difficulty": 6,
        "track_length": 5.278,
        "track_type": "mixed"
    },
    "Japan": {
        "round": 4,
        "date": "2024-04-07",
        "location": {"lat": 34.8431, "lon": 136.5392},
        "track_difficulty": 9,
        "track_length": 5.807,
        "track_type": "technical"
    },
    "China": {
        "round": 5,
        "date": "2024-04-21",
        "location": {"lat": 31.3389, "lon": 121.2208},
        "track_difficulty": 7,
        "track_length": 5.451,
        "track_type": "mixed"
    },
    "Miami": {
        "round": 6,
        "date": "2024-05-05",
        "location": {"lat": 25.9581, "lon": -80.2389},
        "track_difficulty": 6,
        "track_length": 5.412,
        "track_type": "street"
    },
    "Emilia Romagna": {
        "round": 7,
        "date": "2024-05-19",
        "location": {"lat": 44.3439, "lon": 11.7167},
        "track_difficulty": 8,
        "track_length": 4.909,
        "track_type": "technical"
    },
    "Monaco": {
        "round": 8,
        "date": "2024-05-26",
        "location": {"lat": 43.7347, "lon": 7.4206},
        "track_difficulty": 10,
        "track_length": 3.337,
        "track_type": "street"
    },
    "Canada": {
        "round": 9,
        "date": "2024-06-09",
        "location": {"lat": 45.5000, "lon": -73.5228},
        "track_difficulty": 7,
        "track_length": 4.361,
        "track_type": "mixed"
    },
    "Spain": {
        "round": 10,
        "date": "2024-06-23",
        "location": {"lat": 41.5700, "lon": 2.2611},
        "track_difficulty": 7,
        "track_length": 4.675,
        "track_type": "smooth"
    },
    "Austria": {
        "round": 11,
        "date": "2024-06-30",
        "location": {"lat": 47.2197, "lon": 14.7647},
        "track_difficulty": 6,
        "track_length": 4.318,
        "track_type": "smooth"
    },
    "Great Britain": {
        "round": 12,
        "date": "2024-07-07",
        "location": {"lat": 52.0786, "lon": -1.0169},
        "track_difficulty": 8,
        "track_length": 5.891,
        "track_type": "technical"
    },
    "Hungary": {
        "round": 13,
        "date": "2024-07-21",
        "location": {"lat": 47.5829, "lon": 19.2486},
        "track_difficulty": 7,
        "track_length": 4.381,
        "track_type": "technical"
    },
    "Belgium": {
        "round": 14,
        "date": "2024-07-28",
        "location": {"lat": 50.4372, "lon": 5.9714},
        "track_difficulty": 9,
        "track_length": 7.004,
        "track_type": "technical"
    },
    "Netherlands": {
        "round": 15,
        "date": "2024-08-25",
        "location": {"lat": 52.3888, "lon": 4.5409},
        "track_difficulty": 7,
        "track_length": 4.259,
        "track_type": "smooth"
    },
    "Italy": {
        "round": 16,
        "date": "2024-09-01",
        "location": {"lat": 45.6206, "lon": 9.2811},
        "track_difficulty": 6,
        "track_length": 5.793,
        "track_type": "smooth"
    },
    "Azerbaijan": {
        "round": 17,
        "date": "2024-09-15",
        "location": {"lat": 40.3725, "lon": 49.8533},
        "track_difficulty": 8,
        "track_length": 6.003,
        "track_type": "street"
    },
    "Singapore": {
        "round": 18,
        "date": "2024-09-22",
        "location": {"lat": 1.2914, "lon": 103.8640},
        "track_difficulty": 9,
        "track_length": 5.063,
        "track_type": "street"
    },
    "USA": {
        "round": 19,
        "date": "2024-10-20",
        "location": {"lat": 30.1328, "lon": -97.6411},
        "track_difficulty": 7,
        "track_length": 5.513,
        "track_type": "mixed"
    },
    "Mexico": {
        "round": 20,
        "date": "2024-10-27",
        "location": {"lat": 19.4042, "lon": -99.0907},
        "track_difficulty": 6,
        "track_length": 4.304,
        "track_type": "smooth"
    },
    "Brazil": {
        "round": 21,
        "date": "2024-11-03",
        "location": {"lat": -23.7036, "lon": -46.6997},
        "track_difficulty": 7,
        "track_length": 4.309,
        "track_type": "technical"
    },
    "Las Vegas": {
        "round": 22,
        "date": "2024-11-23",
        "location": {"lat": 36.1147, "lon": -115.1728},
        "track_difficulty": 6,
        "track_length": 6.201,
        "track_type": "street"
    },
    "Qatar": {
        "round": 23,
        "date": "2024-12-01",
        "location": {"lat": 25.4900, "lon": 51.4542},
        "track_difficulty": 6,
        "track_length": 5.419,
        "track_type": "smooth"
    },
    "Abu Dhabi": {
        "round": 24,
        "date": "2024-12-08",
        "location": {"lat": 24.4672, "lon": 54.6031},
        "track_difficulty": 6,
        "track_length": 5.281,
        "track_type": "smooth"
    }
}

# Driver characteristics for 2024
drivers_2024 = {
    "VER": {"name": "Max Verstappen", "team": "Red Bull", "skill": 95, "wet_skill": 95, "consistency": 92},
    "PER": {"name": "Sergio Perez", "team": "Red Bull", "skill": 86, "wet_skill": 82, "consistency": 80},
    "HAM": {"name": "Lewis Hamilton", "team": "Mercedes", "skill": 93, "wet_skill": 95, "consistency": 91},
    "RUS": {"name": "George Russell", "team": "Mercedes", "skill": 88, "wet_skill": 85, "consistency": 86},
    "LEC": {"name": "Charles Leclerc", "team": "Ferrari", "skill": 91, "wet_skill": 87, "consistency": 85},
    "SAI": {"name": "Carlos Sainz", "team": "Ferrari", "skill": 88, "wet_skill": 85, "consistency": 86},
    "NOR": {"name": "Lando Norris", "team": "McLaren", "skill": 90, "wet_skill": 88, "consistency": 87},
    "PIA": {"name": "Oscar Piastri", "team": "McLaren", "skill": 86, "wet_skill": 83, "consistency": 82},
    "ALO": {"name": "Fernando Alonso", "team": "Aston Martin", "skill": 92, "wet_skill": 93, "consistency": 89},
    "STR": {"name": "Lance Stroll", "team": "Aston Martin", "skill": 82, "wet_skill": 80, "consistency": 78},
    "OCO": {"name": "Esteban Ocon", "team": "Alpine", "skill": 83, "wet_skill": 81, "consistency": 81},
    "GAS": {"name": "Pierre Gasly", "team": "Alpine", "skill": 84, "wet_skill": 82, "consistency": 80},
    "ALB": {"name": "Alexander Albon", "team": "Williams", "skill": 85, "wet_skill": 83, "consistency": 82},
    "SAR": {"name": "Logan Sargeant", "team": "Williams", "skill": 76, "wet_skill": 75, "consistency": 74},
    "RIC": {"name": "Daniel Ricciardo", "team": "RB", "skill": 84, "wet_skill": 83, "consistency": 81},
    "TSU": {"name": "Yuki Tsunoda", "team": "RB", "skill": 82, "wet_skill": 80, "consistency": 78},
    "HUL": {"name": "Nico Hulkenberg", "team": "Haas", "skill": 83, "wet_skill": 82, "consistency": 81},
    "MAG": {"name": "Kevin Magnussen", "team": "Haas", "skill": 82, "wet_skill": 81, "consistency": 79},
    "BOT": {"name": "Valtteri Bottas", "team": "Sauber", "skill": 84, "wet_skill": 82, "consistency": 83},
    "ZHO": {"name": "Zhou Guanyu", "team": "Sauber", "skill": 78, "wet_skill": 77, "consistency": 76}
}

# Team characteristics for 2024
teams_2024 = {
    "Red Bull": {"performance": 95, "reliability": 88, "wet_performance": 92},
    "Mercedes": {"performance": 88, "reliability": 92, "wet_performance": 90},
    "Ferrari": {"performance": 91, "reliability": 85, "wet_performance": 86},
    "McLaren": {"performance": 93, "reliability": 88, "wet_performance": 87},
    "Aston Martin": {"performance": 83, "reliability": 87, "wet_performance": 84},
    "Alpine": {"performance": 80, "reliability": 83, "wet_performance": 79},
    "Williams": {"performance": 78, "reliability": 82, "wet_performance": 80},
    "RB": {"performance": 79, "reliability": 85, "wet_performance": 81},
    "Haas": {"performance": 77, "reliability": 80, "wet_performance": 78},
    "Sauber": {"performance": 75, "reliability": 81, "wet_performance": 76}
}

class F1Predictor:
    def __init__(self, api_key, cache_dir='weather_cache'):
        self.api_key = api_key
        self.cache_dir = cache_dir
        self.weather_cache = {}
        self.lap_data = None
        self.performance_metrics = None
        self.model = None

        # Create cache directory if it doesn't exist
        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)

        # Load cached weather data if available
        self._load_weather_cache()

    def _load_weather_cache(self):
        """Load cached weather data from file"""
        cache_file = os.path.join(self.cache_dir, 'weather_cache.json')
        if os.path.exists(cache_file):
            try:
                with open(cache_file, 'r') as f:
                    self.weather_cache = json.load(f)
                print(f"Loaded weather data for {len(self.weather_cache)} races from cache")
            except Exception as e:
                print(f"Error loading weather cache: {e}")
                self.weather_cache = {}

    def _save_weather_cache(self):
        """Save weather data to cache file"""
        cache_file = os.path.join(self.cache_dir, 'weather_cache.json')
        try:
            with open(cache_file, 'w') as f:
                json.dump(self.weather_cache, f)
        except Exception as e:
            print(f"Error saving weather cache: {e}")

    def get_weather_data(self, race_name):
        """Get weather data for a specific race, using cache if available"""
        if race_name in self.weather_cache:
            print(f"Using cached weather data for {race_name}")
            return self.weather_cache[race_name]

        race_info = races_2024[race_name]
        lat = race_info["location"]["lat"]
        lon = race_info["location"]["lon"]
        date = race_info["date"]

        weather = self._fetch_openweather_data(lat, lon, date)

        self.weather_cache[race_name] = weather
        self._save_weather_cache()

        return weather

    def _fetch_openweather_data(self, lat, lon, date_str):
        """Fetch weather data from OpenWeather API 2.5"""
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        today = datetime.now()
        days_diff = (date_obj - today).days

        print(f"Fetching weather data for {date_str} at {lat}, {lon}")

        if days_diff <= 5 and days_diff >= 0:
            url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={self.api_key}&units=metric"
            try:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()

                target_time = date_obj.replace(hour=12, minute=0, second=0)
                forecasts = data.get('list', [])
                closest_forecast = min(
                    forecasts,
                    key=lambda x: abs(datetime.fromtimestamp(x['dt']) - target_time),
                    default=None
                )

                if closest_forecast:
                    return {
                        'main': closest_forecast['weather'][0]['main'],
                        'description': closest_forecast['weather'][0]['description'],
                        'temp': closest_forecast['main']['temp'],
                        'humidity': closest_forecast['main']['humidity'],
                        'wind_speed': closest_forecast['wind']['speed'],
                        'pressure': closest_forecast['main'].get('pressure', 1013),
                        'rain_1h': closest_forecast.get('rain', {}).get('3h', 0) / 3,
                        'clouds': closest_forecast['clouds']['all']
                    }
            except Exception as e:
                print(f"Error fetching forecast: {e}")

        return self._simulate_weather_data(date_obj.month)

    def _simulate_weather_data(self, month):
        """Generate simulated weather data based on month"""
        season_data = {
            12: {"temp_range": (0, 15), "rain_prob": 0.4, "clouds_prob": 0.7},
            1: {"temp_range": (0, 15), "rain_prob": 0.4, "clouds_prob": 0.7},
            2: {"temp_range": (0, 15), "rain_prob": 0.4, "clouds_prob": 0.7},
            3: {"temp_range": (5, 20), "rain_prob": 0.3, "clouds_prob": 0.5},
            4: {"temp_range": (10, 25), "rain_prob": 0.3, "clouds_prob": 0.5},
            5: {"temp_range": (15, 30), "rain_prob": 0.2, "clouds_prob": 0.4},
            6: {"temp_range": (20, 35), "rain_prob": 0.1, "clouds_prob": 0.3},
            7: {"temp_range": (20, 35), "rain_prob": 0.1, "clouds_prob": 0.3},
            8: {"temp_range": (20, 35), "rain_prob": 0.1, "clouds_prob": 0.3},
            9: {"temp_range": (15, 30), "rain_prob": 0.2, "clouds_prob": 0.4},
            10: {"temp_range": (10, 25), "rain_prob": 0.3, "clouds_prob": 0.5},
            11: {"temp_range": (5, 20), "rain_prob": 0.4, "clouds_prob": 0.6},
        }

        data = season_data.get(month, {"temp_range": (15, 25), "rain_prob": 0.3, "clouds_prob": 0.5})

        rain_check = np.random.random()
        cloud_check = np.random.random()

        if rain_check < data["rain_prob"]:
            weather_main = "Rain" if rain_check < data["rain_prob"]/2 else "Drizzle"
            rain_amount = np.random.uniform(0.5, 10)
        else:
            rain_amount = 0
            weather_main = "Clouds" if cloud_check < data["clouds_prob"] else "Clear"

        print("Using simulated weather data")
        return {
            'main': weather_main,
            'description': weather_main.lower(),
            'temp': round(np.random.uniform(data["temp_range"][0], data["temp_range"][1]), 1),
            'humidity': round(np.random.uniform(40, 95)),
            'wind_speed': round(np.random.uniform(1, 15), 1),
            'pressure': round(np.random.uniform(1000, 1025)),
            'rain_1h': rain_amount,
            'clouds': round(np.random.uniform(0, 100)) if weather_main in ["Clouds", "Rain", "Drizzle"] else 0
        }

    def fetch_historical_lap_data(self, year=2024, rounds=None):
        """Fetch historical lap time and qualifying data using FastF1"""
        if rounds is None:
            rounds = [1, 2, 3, 4, 5]

        print(f"Fetching historical lap data for {year}, rounds {rounds}")
        all_lap_data = []

        driver_mapping = {'BEA': 'SAI'}

        for race_round in rounds:
            try:
                race = fastf1.get_session(year, race_round, 'R')
                race.load()
                print(f"Loaded race data for {year} Round {race_round}: {race.event['EventName']}")

                laps = race.laps
                laps['Year'] = year
                laps['Round'] = race_round
                laps['EventName'] = race.event['EventName']

                laps['Driver'] = laps['Driver'].replace(driver_mapping)

                qualy = fastf1.get_session(year, race_round, 'Q')
                qualy.load()
                print(f"Loaded qualifying data for {year} Round {race_round}")
                qualy_results = qualy.results[['DriverNumber', 'Position']].rename(
                    columns={'Position': 'QualifyingPosition'}
                )
                laps = laps.merge(qualy_results, on='DriverNumber', how='left')

                all_lap_data.append(laps)

            except Exception as e:
                print(f"Error loading data for {year} Round {race_round}: {e}")
                continue

        if all_lap_data:
            self.lap_data = pd.concat(all_lap_data)
            unique_drivers = len(self.lap_data['Driver'].unique())
            print(f"Successfully loaded {len(all_lap_data)} races with {unique_drivers} unique drivers")
            if unique_drivers < 18:
                print("Warning: Insufficient driver data, using fallback")
                self.lap_data = None
                return None
            return self.lap_data
        else:
            print("No lap data available, using fallback")
            self.lap_data = None
            return None

    def process_lap_data(self):
        """Process lap data to extract driver performance metrics"""
        if self.lap_data is None or self.lap_data.empty:
            print("No lap data to process")
            self.performance_metrics = None
            return None

        metrics = []
        valid_drivers = set(drivers_2024.keys())
        grouped = self.lap_data[self.lap_data['Driver'].isin(valid_drivers)].groupby(['EventName', 'Driver'])

        for (event, driver_code), group in grouped:
            valid_laps = group[group['LapTime'].notna()]
            if valid_laps.empty:
                continue

            valid_laps['LapTimeSeconds'] = valid_laps['LapTime'].dt.total_seconds()

            median_lap = valid_laps['LapTimeSeconds'].median()
            fastest_lap = valid_laps['LapTimeSeconds'].min()
            lap_std = valid_laps['LapTimeSeconds'].std()

            if pd.isna(lap_std):
                lap_std = 0

            consistency = max(0, 100 - (lap_std * 5))

            if 'Position' in valid_laps.columns:
                final_pos = valid_laps.iloc[-1]['Position']
                start_pos = valid_laps.iloc[0]['Position']
                pos_change = start_pos - final_pos
            else:
                final_pos = np.nan
                start_pos = np.nan
                pos_change = 0

            qualifying_pos = valid_laps['QualifyingPosition'].iloc[0] if 'QualifyingPosition' in valid_laps.columns and not valid_laps['QualifyingPosition'].isna().all() else np.nan

            driver_metrics = {
                'Driver': driver_code,
                'Event': event,
                'Round': valid_laps['Round'].iloc[0],
                'MedianLapTime': median_lap,
                'FastestLapTime': fastest_lap,
                'LapTimeStd': lap_std,
                'Consistency': consistency,
                'FinalPosition': final_pos,
                'StartingPosition': start_pos,
                'QualifyingPosition': qualifying_pos,
                'PositionChange': pos_change,
                'LapsCompleted': len(valid_laps)
            }

            metrics.append(driver_metrics)

        self.performance_metrics = pd.DataFrame(metrics)
        if self.performance_metrics.empty:
            print("No valid performance metrics processed")
            return None
        return self.performance_metrics

    def _calculate_recent_form(self):
        """Calculate weighted average position in last 3 races, blended with skill"""
        race_results = {
            "Bahrain": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "RUS": 5,
                "HAM": 6, "NOR": 7, "PIA": 8, "ALO": 9, "STR": 10,
                "OCO": 11, "GAS": 12, "ALB": 13, "MAG": 14, "TSU": 15,
                "RIC": 16, "HUL": 17, "ZHO": 18, "SAR": 19, "BOT": 20
            },
            "Saudi Arabia": {
                "VER": 1, "PER": 2, "LEC": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "SAI": 7, "HAM": 8, "HUL": 9, "MAG": 10,
                "ALB": 11, "STR": 12, "RIC": 13, "GAS": 14, "OCO": 15,
                "SAR": 16, "TSU": 17, "BOT": 18, "ZHO": 19, "NOR": 20
            },
            "Australia": {
                "SAI": 1, "LEC": 2, "NOR": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "HAM": 7, "TSU": 8, "HUL": 9, "MAG": 10,
                "RIC": 11, "PER": 12, "ALB": 13, "OCO": 14, "BOT": 15,
                "ZHO": 16, "GAS": 17, "STR": 18, "VER": 19, "SAR": 20
            },
            "Japan": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "NOR": 5,
                "ALO": 6, "RUS": 7, "PIA": 8, "HAM": 9, "TSU": 10,
                "HUL": 11, "STR": 12, "RIC": 13, "OCO": 14, "ALB": 15,
                "MAG": 16, "GAS": 17, "BOT": 18, "ZHO": 19, "SAR": 20
            },
            "China": {
                "VER": 1, "NOR": 2, "PER": 3, "LEC": 4, "SAI": 5,
                "RUS": 6, "ALO": 7, "PIA": 8, "HAM": 9, "HUL": 10,
                "OCO": 11, "ALB": 12, "RIC": 13, "STR": 14, "GAS": 15,
                "MAG": 16, "BOT": 17, "ZHO": 18, "TSU": 19, "SAR": 20
            }
        }

        recent_form = {}
        race_order = ["China", "Japan", "Australia", "Saudi Arabia", "Bahrain"]

        for driver_code in drivers_2024:
            skill = drivers_2024[driver_code]["skill"]
            skill_based_form = 21 - (skill / 10)

            positions = []
            if self.performance_metrics is not None and not self.performance_metrics.empty:
                driver_races = self.performance_metrics[self.performance_metrics['Driver'] == driver_code]
                sorted_races = driver_races.sort_values('Round', ascending=False).head(3)
                positions = sorted_races['FinalPosition'].dropna().tolist()

            if len(positions) < 2:
                positions = []
                for race in race_order[:3]:
                    if race in race_results and driver_code in race_results[race]:
                        positions.append(race_results[race][driver_code])

            if not positions:
                recent_form[driver_code] = skill_based_form
                continue

            weights = [0.6, 0.3, 0.1][:len(positions)]
            weights += [0] * (3 - len(weights))
            weighted_avg = np.average(positions, weights=weights)
            blended_form = 0.8 * weighted_avg + 0.2 * skill_based_form
            blended_form = max(blended_form, skill_based_form * 0.99)
            recent_form[driver_code] = blended_form if not np.isnan(blended_form) else skill_based_form

        return recent_form

    def _calculate_track_type_performance(self, target_track_type):
        """Calculate average position on tracks of the same type, blended with team"""
        if self.performance_metrics is None or self.performance_metrics.empty:
            return {driver: 10 for driver in drivers_2024}

        monaco_skill_adjust = {
            'LEC': 0.8, 'VER': 0.85, 'NOR': 0.9, 'HAM': 0.9, 'SAI': 0.9,
            'PER': 0.95, 'RUS': 0.95, 'PIA': 0.98, 'ALO': 0.98, 'RIC': 0.98,
            'OCO': 1.0, 'GAS': 1.0, 'ALB': 1.0, 'HUL': 1.02, 'BOT': 1.02,
            'TSU': 1.05, 'STR': 1.1, 'MAG': 1.1, 'ZHO': 1.1, 'SAR': 1.15
        }

        track_type_perf = {}
        relevant_races = [name for name, info in races_2024.items() if info['track_type'] == target_track_type]

        for driver_code in drivers_2024:
            driver_races = self.performance_metrics[
                (self.performance_metrics['Driver'] == driver_code) &
                (self.performance_metrics['Event'].isin(relevant_races))
            ]
            valid_positions = driver_races['FinalPosition'].dropna()
            team_perf = teams_2024[drivers_2024[driver_code]["team"]]["performance"]

            team_weight = team_perf / 100 * 0.3 + 0.85
            if len(valid_positions) < 3:
                base_perf = 21 - (team_perf / 10)
                if target_track_type == 'street':
                    base_perf *= monaco_skill_adjust.get(driver_code, 1.0)
                track_type_perf[driver_code] = min(base_perf * 1.2, 15)
                continue

            avg_position = valid_positions.mean() / team_weight
            blended_perf = 0.8 * avg_position + 0.2 * (21 - (team_perf / 10))
            if target_track_type == 'street':
                blended_perf *= monaco_skill_adjust.get(driver_code, 1.0)
            track_type_perf[driver_code] = min(blended_perf, 12) if not np.isnan(blended_perf) else 10

        return track_type_perf

    def prepare_training_data(self, race_rounds=None):
        """Prepare training data from lap metrics and race information"""
        if race_rounds is None:
            race_rounds = [1, 2, 3, 4, 5]

        race_names = [race for race, info in races_2024.items() if info["round"] in race_rounds]

        print(f"Preparing training data for races: {', '.join(race_names)}")

        if self.performance_metrics is None and self.lap_data is not None:
            self.process_lap_data()

        if self.performance_metrics is None or self.performance_metrics.empty:
            return self._create_basic_training_data(race_names)

        training_data = []
        recent_form = self._calculate_recent_form()

        for race_name in race_names:
            race_info = races_2024[race_name]
            weather = self.get_weather_data(race_name)
            track_type_perf = self._calculate_track_type_performance(race_info["track_type"])

            is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
            is_cloudy = 1 if weather["main"] == "Clouds" else 0
            temperature = weather["temp"]
            humidity = weather["humidity"]
            wind_speed = weather["wind_speed"]

            for driver_code, driver_info in drivers_2024.items():
                team_info = teams_2024[driver_info["team"]]

                driver_race_metrics = self.performance_metrics[
                    (self.performance_metrics['Driver'] == driver_code) &
                    (self.performance_metrics['Event'].str.contains(race_name))
                ]

                if not driver_race_metrics.empty:
                    consistency = driver_race_metrics['Consistency'].iloc[0]
                    finish_position = driver_race_metrics['FinalPosition'].iloc[0]
                    qualifying_position = driver_race_metrics['QualifyingPosition'].iloc[0] if 'QualifyingPosition' in driver_race_metrics.columns and not pd.isna(driver_race_metrics['QualifyingPosition'].iloc[0]) else 10
                    if pd.isna(finish_position):
                        finish_position = 20
                else:
                    consistency = driver_info["consistency"]
                    finish_position = 20
                    qualifying_position = 10

                features = {
                    'Driver': driver_code,
                    'DriverName': driver_info["name"],
                    'Race': race_name,
                    'TrackDifficulty': race_info["track_difficulty"],
                    'TrackLength': race_info["track_length"],
                    'TrackType': race_info["track_type"],
                    'TeamPerformance': team_info["performance"],
                    'TeamReliability': team_info["reliability"],
                    'TeamWetPerformance': team_info["wet_performance"],
                    'DriverSkill': driver_info["skill"],
                    'DriverWetSkill': driver_info["wet_skill"],
                    'DriverConsistency': consistency,
                    'RecentForm': recent_form.get(driver_code, 10),
                    'TrackTypePerformance': track_type_perf.get(driver_code, 10),
                    'QualifyingPosition': qualifying_position,
                    'IsWet': is_wet,
                    'IsCloudy': is_cloudy,
                    'Temperature': temperature,
                    'Humidity': humidity,
                    'WindSpeed': wind_speed,
                    'Position': finish_position
                }

                training_data.append(features)

        return pd.DataFrame(training_data)

    def _create_basic_training_data(self, race_names):
        """Create basic training data without lap metrics"""
        print("Creating basic training data from known results")
        training_data = []

        race_results = {
            "Bahrain": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "RUS": 5,
                "HAM": 6, "NOR": 7, "PIA": 8, "ALO": 9, "STR": 10,
                "OCO": 11, "GAS": 12, "ALB": 13, "MAG": 14, "TSU": 15,
                "RIC": 16, "HUL": 17, "ZHO": 18, "SAR": 19, "BOT": 20
            },
            "Saudi Arabia": {
                "VER": 1, "PER": 2, "LEC": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "SAI": 7, "HAM": 8, "HUL": 9, "MAG": 10,
                "ALB": 11, "STR": 12, "RIC": 13, "GAS": 14, "OCO": 15,
                "SAR": 16, "TSU": 17, "BOT": 18, "ZHO": 19, "NOR": 20
            },
            "Australia": {
                "SAI": 1, "LEC": 2, "NOR": 3, "PIA": 4, "ALO": 5,
                "RUS": 6, "HAM": 7, "TSU": 8, "HUL": 9, "MAG": 10,
                "RIC": 11, "PER": 12, "ALB": 13, "OCO": 14, "BOT": 15,
                "ZHO": 16, "GAS": 17, "STR": 18, "VER": 19, "SAR": 20
            },
            "Japan": {
                "VER": 1, "PER": 2, "SAI": 3, "LEC": 4, "NOR": 5,
                "ALO": 6, "RUS": 7, "PIA": 8, "HAM": 9, "TSU": 10,
                "HUL": 11, "STR": 12, "RIC": 13, "OCO": 14, "ALB": 15,
                "MAG": 16, "GAS": 17, "BOT": 18, "ZHO": 19, "SAR": 20
            },
            "China": {
                "VER": 1, "NOR": 2, "PER": 3, "LEC": 4, "SAI": 5,
                "RUS": 6, "ALO": 7, "PIA": 8, "HAM": 9, "HUL": 10,
                "OCO": 11, "ALB": 12, "RIC": 13, "STR": 14, "GAS": 15,
                "MAG": 16, "BOT": 17, "ZHO": 18, "TSU": 19, "SAR": 20
            }
        }

        recent_form = self._calculate_recent_form()

        for race_name in race_names:
            race_info = races_2024[race_name]
            weather = self.get_weather_data(race_name)
            track_type_perf = self._calculate_track_type_performance(race_info["track_type"])

            is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
            is_cloudy = 1 if weather["main"] == "Clouds" else 0
            temperature = weather["temp"]
            humidity = weather["humidity"]
            wind_speed = weather["wind_speed"]

            for driver_code, driver_info in drivers_2024.items():
                team_info = teams_2024[driver_info["team"]]

                if race_name in race_results and driver_code in race_results[race_name]:
                    position = race_results[race_name][driver_code]
                    qualifying_position = max(1, min(20, position + np.random.randint(-2, 3)))
                else:
                    position = 21 - round((team_info["performance"] + driver_info["skill"]) / 20)
                    position = max(1, min(20, position))
                    qualifying_position = max(1, min(20, position + np.random.randint(-2, 3)))

                features = {
                    'Driver': driver_code,
                    'DriverName': driver_info["name"],
                    'Race': race_name,
                    'TrackDifficulty': race_info["track_difficulty"],
                    'TrackLength': race_info["track_length"],
                    'TrackType': race_info["track_type"],
                    'TeamPerformance': team_info["performance"],
                    'TeamReliability': team_info["reliability"],
                    'TeamWetPerformance': team_info["wet_performance"],
                    'DriverSkill': driver_info["skill"],
                    'DriverWetSkill': driver_info["wet_skill"],
                    'DriverConsistency': driver_info["consistency"],
                    'RecentForm': recent_form.get(driver_code, 10),
                    'TrackTypePerformance': track_type_perf.get(driver_code, 10),
                    'QualifyingPosition': qualifying_position,
                    'IsWet': is_wet,
                    'IsCloudy': is_cloudy,
                    'Temperature': temperature,
                    'Humidity': humidity,
                    'WindSpeed': wind_speed,
                    'Position': position
                }

                training_data.append(features)

        return pd.DataFrame(training_data)

    def prepare_prediction_data(self, race_name):
        """Prepare prediction data for a specific race"""
        print(f"Preparing prediction data for {race_name}")
        race_info = races_2024[race_name]

        weather = self.get_weather_data(race_name)

        is_wet = 1 if weather["main"] in ["Rain", "Drizzle"] else 0
        is_cloudy = 1 if weather["main"] == "Clouds" else 0
        temperature = weather["temp"]
        humidity = weather["humidity"]
        wind_speed = weather["wind_speed"]

        print(f"\nPrediction for: {race_name} Grand Prix")
        print(f"Date: {race_info['date']}")
        print(f"Weather: {weather['main']} ({weather['description']})")
        print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind: {wind_speed} m/s")
        print(f"Track difficulty: {race_info['track_difficulty']}/10, Type: {race_info['track_type']}")

        prediction_data = []
        recent_form = self._calculate_recent_form()
        track_type_perf = self._calculate_track_type_performance(race_info["track_type"])

        for driver_code, driver_info in drivers_2024.items():
            team_info = teams_2024[driver_info["team"]]

            qual_scores = []
            for dc, di in drivers_2024.items():
                ti = teams_2024[di["team"]]
                score = (ti["performance"] * 0.3 + di["skill"] * 0.7) / 5
                qual_scores.append((dc, score, di["skill"], di["consistency"]))
            qual_scores.sort(key=lambda x: (x[1], x[2], x[3]), reverse=True)
            qualifying_position = next(i + 1 for i, (dc, _, _, _) in enumerate(qual_scores) if dc == driver_code)
            qualifying_position = max(1, min(20, qualifying_position + np.random.randint(0, 2)))

            if driver_code in ['VER', 'LEC', 'SAI', 'RUS', 'NOR', 'PIA', 'SAR', 'TSU', 'ALO', 'HUL']:
                print(f"Driver {driver_info['name']}: QualifyingPosition={qualifying_position}, RecentForm={recent_form.get(driver_code, 10):.2f}, TrackTypePerformance={track_type_perf.get(driver_code, 10):.2f}, DriverSkill={driver_info['skill']}, DriverConsistency={driver_info['consistency']}")

            features = {
                'Driver': driver_code,
                'DriverName': driver_info["name"],
                'Race': race_name,
                'TrackDifficulty': race_info["track_difficulty"],
                'TrackLength': race_info["track_length"],
                'TrackType': race_info["track_type"],
                'TeamPerformance': team_info["performance"],
                'TeamReliability': team_info["reliability"],
                'TeamWetPerformance': team_info["wet_performance"],
                'DriverSkill': driver_info["skill"],
                'DriverWetSkill': driver_info["wet_skill"],
                'DriverConsistency': driver_info["consistency"],
                'RecentForm': recent_form.get(driver_code, 10),
                'TrackTypePerformance': track_type_perf.get(driver_code, 10),
                'QualifyingPosition': qualifying_position,
                'IsWet': is_wet,
                'IsCloudy': is_cloudy,
                'Temperature': temperature,
                'Humidity': humidity,
                'WindSpeed': wind_speed
            }

            prediction_data.append(features)

        return pd.DataFrame(prediction_data)

    def train_model(self, training_data):
        """Train the Random Forest model"""
        print("Training Random Forest model...")

        numeric_features = [
            'TrackDifficulty', 'TrackLength', 'TeamPerformance',
            'TeamReliability', 'TeamWetPerformance', 'DriverSkill',
            'DriverWetSkill', 'DriverConsistency', 'RecentForm',
            'TrackTypePerformance', 'QualifyingPosition', 'Temperature',
            'Humidity', 'WindSpeed'
        ]
        categorical_features = ['TrackType', 'IsWet', 'IsCloudy']

        X = training_data.drop(['Driver', 'DriverName', 'Race', 'Position'], axis=1, errors='ignore')
        y = training_data['Position']

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), [f for f in numeric_features if f not in ['DriverSkill', 'TeamPerformance', 'QualifyingPosition', 'DriverWetSkill', 'TeamWetPerformance', 'TrackTypePerformance', 'DriverConsistency', 'RecentForm']]),
                ('skill', StandardScaler(), ['DriverSkill']),
                ('team', StandardScaler(), ['TeamPerformance']),
                ('qualy', StandardScaler(), ['QualifyingPosition']),
                ('wet_skill', StandardScaler(), ['DriverWetSkill']),
                ('wet_team', StandardScaler(), ['TeamWetPerformance']),
                ('track_type', StandardScaler(), ['TrackTypePerformance']),
                ('consistency', StandardScaler(), ['DriverConsistency']),
                ('form', StandardScaler(), ['RecentForm']),
                ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
            ],
            transformer_weights={'num': 1.0, 'skill': 1.2, 'team': 1.2, 'qualy': 2.0, 'wet_skill': 0.5, 'wet_team': 0.5, 'track_type': 1.5, 'consistency': 1.8, 'form': 2.0, 'cat': 1.0}
        )

        self.model = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', RandomForestRegressor(
                n_estimators=200,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42
            ))
        ])

        self.model.fit(X, y)
        print("Model training completed")

        return self.model

    def predict_race(self, race_name):
        """Predict race results for a given race"""
        if self.model is None:
            raise ValueError("Model not trained. Please train the model first.")

        pred_data = self.prepare_prediction_data(race_name)

        X_pred = pred_data.drop(['Driver', 'DriverName', 'Race'], axis=1)
        predictions = self.model.predict(X_pred)

        results = pd.DataFrame({
            'Driver': pred_data['Driver'],
            'DriverName': pred_data['DriverName'],
            'Team': [drivers_2024[driver]['team'] for driver in pred_data['Driver']],
            'PredictedPosition': predictions
        })

        results = results.sort_values('PredictedPosition')
        results['PredictedPosition'] = range(1, len(results) + 1)

        return results

    def visualize_predictions(self, race_name, results):
        """Visualize the predicted race results"""
        plt.figure(figsize=(12, 8))

        top_10 = results.head(10)

        bars = plt.bar(top_10['DriverName'], top_10['PredictedPosition'])

        plt.title(f'Predicted Race Results for {race_name} Grand Prix')
        plt.xlabel('Driver')
        plt.ylabel('Predicted Position')
        plt.xticks(rotation=45, ha='right')

        plt.gca().invert_yaxis()
        plt.yticks(range(1, 11))
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)

        team_colors = {
            'Red Bull': '#0600EF',
            'Mercedes': '#00D2BE',
            'Ferrari': '#DC0000',
            'McLaren': '#FF8700',
            'Aston Martin': '#00665E',
            'Alpine': '#0090FF',
            'Williams': '#005AFF',
            'RB': '#2B4562',
            'Haas': '#FFFFFF',
            'Sauber': '#0066FF'
        }

        for bar, team in zip(bars, top_10['Team']):
            bar.set_color(team_colors.get(team, '#888888'))

        plt.tight_layout()
        plt.savefig('race_prediction.png')
        plt.close()

    def run_prediction_pipeline(self, train_rounds=None):
        """Run the complete prediction pipeline for a randomly selected race"""
        print(f"\nRunning prediction pipeline...")

        if train_rounds is None:
            train_rounds = [1, 2, 3, 4, 5]

        # Select a random race not in training rounds
        available_races = [name for name, info in races_2024.items() if info["round"] not in train_rounds]
        race_name = random.choice(available_races)
        print(f"Selected race: {race_name} (Round {races_2024[race_name]['round']})")

        self.fetch_historical_lap_data(rounds=train_rounds)
        training_data = self.prepare_training_data(train_rounds)

        self.train_model(training_data)

        results = self.predict_race(race_name)

        print(f"\nPredicted Results for {race_name} Grand Prix:")
        print("\nPosition | Driver | Team")
        print("-" * 40)
        for idx, row in results.head(10).iterrows():
            print(f"{row['PredictedPosition']:2d} | {row['DriverName']:<20} | {row['Team']}")

        self.visualize_predictions(race_name, results)

        return results

if __name__ == "__main__":
    predictor = F1Predictor(api_key=OPENWEATHER_API_KEY)

    training_rounds = [1, 2, 3, 4, 5]

    try:
        results = predictor.run_prediction_pipeline(training_rounds)
    except Exception as e:
        print(f"Error running prediction pipeline: {e}")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
INFO:fastf1.fastf1.req:Using cached data for timing_app_data
core         

Loaded weather data for 8 races from cache

Running prediction pipeline...
Selected race: Qatar (Round 23)
Fetching historical lap data for 2024, rounds [1, 2, 3, 4, 5]


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            I

Loaded race data for 2024 Round 1: Bahrain Grand Prix


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '31', '10']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            I

...

Loaded qualifying data for 2024 Round 4


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:08.313000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '11', '16', '55', '63', '14', '81', '44', '27', '31', '23', '10', '24', '18', '20', '2', '3', '22', '77']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '4', '11', '16', '55', '63', '14', '81', '44', '27', '31', '23', '10', '24', '18', '20', '2', '3', '22', '77']
core           INFO 	Loading data for Chinese Grand Prix - Qualify

Loaded race data for 2024 Round 5: Chinese Grand Prix


req            INFO 	Using cached data for car_data
INFO:fastf1.fastf1.req:Using cached data for car_data
req            INFO 	Using cached data for position_data
INFO:fastf1.fastf1.req:Using cached data for position_data
req            INFO 	Using cached data for weather_data
INFO:fastf1.fastf1.req:Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
INFO:fastf1.fastf1.req:Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '4', '81', '16', '55', '63', '27', '77', '18', '3', '31', '23', '10', '24', '20', '44', '22', '2']
INFO:fastf1.fastf1.core:Finished loading data for 20 drivers: ['1', '11', '14', '4', '81', '16', '55', '63', '27', '77', '18', '3', '31', '23', '10', '24', '20', '44', '22', '2']


Loaded qualifying data for 2024 Round 5
Successfully loaded 5 races with 20 unique drivers
Preparing training data for races: Bahrain, Saudi Arabia, Australia, Japan, China
Using cached weather data for Bahrain
Using cached weather data for Saudi Arabia
Using cached weather data for Australia
Using cached weather data for Japan
Using cached weather data for China
Training Random Forest model...
Model training completed
Preparing prediction data for Qatar
Fetching weather data for 2024-12-01 at 25.49, 51.4542
Using simulated weather data

Prediction for: Qatar Grand Prix
Date: 2024-12-01
Weather: Rain (rain)
Temperature: 14.2°C, Humidity: 78%, Wind: 8.8 m/s
Track difficulty: 6/10, Type: smooth
Driver Max Verstappen: QualifyingPosition=2, RecentForm=11.38, TrackTypePerformance=13.80, DriverSkill=95, DriverConsistency=92
Driver George Russell: QualifyingPosition=10, RecentForm=12.08, TrackTypePerformance=14.64, DriverSkill=88, DriverConsistency=86
Driver Charles Leclerc: QualifyingPositio